In [29]:
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
size = 256

In [30]:
# 匯入資料
x_train,y_train = [],[]
height = pd.read_csv('train.csv')
for index in range(len(height.index)):
    #print(index)
    img = cv2.resize(cv2.imread('train_images/%s'%str(height.iloc[index,0]),0),(size,size))
    x_train.append(img)
    y_train.append(height.iloc[index,1])

np.reshape(y_train,(len(x_train),1))

array([[0],
       [1],
       [1],
       ...,
       [3],
       [0],
       [5]], dtype=int64)

In [31]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(256, 256)),
  tf.keras.layers.Dense(256, activation='relu'),  
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(32, activation='relu'),
  #tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 65536)             0         
                                                                 
 dense_16 (Dense)            (None, 256)               16777472  
                                                                 
 dropout_12 (Dropout)        (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               32896     
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 32)                4128      
                                                                 
 dropout_14 (Dropout)        (None, 32)               

In [32]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1,momentum=0.9,nesterov=True),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
x_train = np.array(x_train,dtype=np.float32)
y_train = np.array(y_train,dtype=np.float32)
# print(train_images)
# print(train_images/255.0)
print(x_train.shape)
print(y_train.shape)

(2528, 256, 256)
(2528,)


In [34]:
history = model.fit(x_train, y_train, epochs=110,validation_split=0.7,batch_size=43)
model.save('keras_model_%s.h5'%size)

Epoch 1/110
18/18 [==============================] - 2s 110ms/step - loss: nan - accuracy: 0.2559 - val_loss: nan - val_accuracy: 0.2689
Epoch 2/110
18/18 [==============================] - 2s 98ms/step - loss: nan - accuracy: 0.2612 - val_loss: nan - val_accuracy: 0.2689
Epoch 3/110
18/18 [==============================] - 2s 104ms/step - loss: nan - accuracy: 0.2612 - val_loss: nan - val_accuracy: 0.2689
Epoch 4/110
18/18 [==============================] - 2s 95ms/step - loss: nan - accuracy: 0.2612 - val_loss: nan - val_accuracy: 0.2689
Epoch 5/110
18/18 [==============================] - 2s 94ms/step - loss: nan - accuracy: 0.2612 - val_loss: nan - val_accuracy: 0.2689
Epoch 6/110
18/18 [==============================] - 2s 92ms/step - loss: nan - accuracy: 0.2612 - val_loss: nan - val_accuracy: 0.2689
Epoch 7/110
18/18 [==============================] - 2s 98ms/step - loss: nan - accuracy: 0.2612 - val_loss: nan - val_accuracy: 0.2689
Epoch 8/110
18/18 [===========================

In [35]:
with tf.device('/gpu:0'):
    train_loss, train_acc = model.evaluate(x_train,  y_train)
print('test_loss:', train_loss)
print('test_acc:', train_acc)

79/79 [==============================] - 2s 19ms/step - loss: nan - accuracy: 0.2666
test_loss: nan
test_acc: 0.2666139304637909


In [36]:
from keras.models import load_model
import csv,pandas as pd,cv2,numpy as np
model = load_model('keras_model_%d.h5'%size) #載入訓練好的模型
# Load Test Data
test_images = []
labels = []
height = pd.read_csv('test.csv')
with tf.device('/gpu:1'):
    for index in range(len(height.index)):    
        img = cv2.resize(cv2.imread('test_images/%s'%(str(height.iloc[index, 0])),-1),(size,size))
        test_images.append(str(height.iloc[index, 0]))
        result = model.predict(img.reshape(-1,size,size,1))
        labels.append(np.argmax(result))

with open('testkeras.csv','w',newline='') as file2:
    filename = ['ID','Label']
    write = csv.DictWriter(file2,fieldnames=filename)
    write.writeheader()
    for i,j in enumerate(labels):
        if i%1000 == 0:
            print(i,j)
        write.writerow({"ID": test_images[i],'Label': j})
                

1/1 [==============================] - 0s 15ms/step
0 0
1000 0
2000 0
3000 0
4000 0
5000 0
6000 0
7000 0
8000 0
9000 0
10000 0
